Будем использовать реализацию Word2vec в библиотеке **Gensim**, а в качестве предобученных моделей возьмем модели Андрея Кутузова и Лизы Кузьменко с сайта [RusVectōrēs.](https://rusvectores.org/ru/models/). 

In [1]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

В качестве моделей давайте возьмем 

1) araneum_none_fasttextcbow_300_5_2018 (fasttext) - модель, обученная на интернет-корпусе русского языка


2) ruscorpora_upos_skipgram_300_5_2018 (word2vec) - модель, обученная НКРЯ

## word2vec + fasttext

Существуют несколько форматов, в которых могут храниться модели - .vec и .model 

1) Первый формат считается классическим вариантом модели word2vec. Для загрузки таакой модели надо воспользоваться методом *KeyedVectors.load_word2vec_format*. 
Модель может быть бинарной, для ее загрузки надо передать параметр binary, равный True. 

2) Формат .model - собственный формат gensim. Такую модель надо загружать с помощью метода *KeyedVectors.load*.

 **1) если модель без тэгов**

In [2]:
# загрузка модели

model_file = 'araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)


#проверка наличия слова в словаре

lemma = 'заграница'
lemma in model

True

In [3]:
model

## Получение вектора документа

Отлично, вектора для слов получены. Что с ними делать дальше? 

Есть два подхода (а точнее есть один, а второй мы придумали, потому что с одним жить нельзя).
> Классика - для получения вектора документа нужно взять и усреднить все вектора его слов
 
$$ vec_{doc} = \frac {\sum_{i=0}^{n} vec_i}{len(d)} $$


In [17]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный', 'замок']

# создаем вектор-маску
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model:
        lemmas_vectors[idx] = model[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


> Эксперимент - представим документ не в виде одного усредненного вектора, а как матрицу векторов входящих в него слов

```
 слово1 |  v1_300
 слово2 |  v2_300
 слово3 |  v3_300
 слово4 |  v4_300
```

> Отлично, теперь каждый документ представлен в виде матрицы векторов своих слов. Но нам надо получить близость матрицы документа в коллекции и матрицы входящего запроса. Как? Умножим две матрицы друг на друга - одна матрица размером d x 300, другая q x 300 - получим попарную близость слов из каждого документа - матрицу размером d x q.


In [19]:
# возьмем игрушечный пример кейса

text1 = 'турция' 
text2 = 'нужна справка срочно'
query = 'быстрая справка'

In [22]:
# построим матрицы всех документов

def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model:
            lemmas_vectors[idx] = model[lemma]
            
    return lemmas_vectors    


text1_m = create_doc_matrix(text1)
text2_m = create_doc_matrix(text2)
query_m = create_doc_matrix(query)

In [23]:
# размер матрицы как и ожидали
query_m.shape

(2, 300)

In [24]:
# посмотрим на близость слов первого текста и слов запроса
text1_m.dot(query_m.T)

array([[0.02087945, 0.01183069]])

In [25]:
# посмотрим на близость слов второго текста и слов запроса
text2_m.dot(query_m.T)

array([[-0.00173893,  0.0355643 ],
       [ 0.00292079,  1.00000011],
       [ 0.04900997,  0.33582122]])

In [26]:
docs_m = [text1_m, text2_m]

    
def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    print(sims)
    return np.argmax(sims)


search(docs_m, query_m)

[0.032710136941469445, 1.0490100806379348]


1

## Задание

Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  

## токенизаторы

In [4]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsNERTagger,

    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

In [5]:
import re
from razdel import tokenize, sentenize
from nltk.corpus import stopwords
stops = stopwords.words("russian")
stops.extend(['здравствовать', 'добрый', 'день', 'спасибо', 'пожалуйста'])

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

def tokenizer(text_data):
    tokens = [_.text for _ in list(tokenize(str(text_data).lower())) if not re.search('[^а-яА-ЯёЁa-zA-z]', _.text)]
    return " ".join(tokens)

def lemmatizer(tokens):
    lem_text = []
    for word in tokens.split(" "):
        lem = morph.parse(word)[0].normal_form
        if lem not in stops:
            lem_text.append(lem)
    return ' '.join(lem_text)

In [6]:
from tqdm.auto import tqdm
tqdm.pandas()

In [7]:
def preprocess_no_NE(text: str) -> str:
    #text = text.replace('\n', ' ')
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        text = text.replace(span.text, "")
    return lemmatizer(tokenizer(text)) #re.sub(r"([^\w\s])", r" \1 ", new_text) 

def preprocess_with_NE(text_data: str) -> str:
    tokens = [_.text for _ in list(tokenize(str(text_data).lower())) if not re.search('[^а-яА-ЯёЁa-zA-z]', _.text)]
    return lemmatizer(" ".join(tokens))

## загрузка данных

In [8]:
import pandas as pd

In [9]:
queries_data = pd.read_excel("queries_base.xlsx").fillna('none')

queries_data["with_NE"] = queries_data["Текст вопроса"].progress_apply(preprocess_with_NE)
queries_data["no_NE"] = queries_data["Текст вопроса"].progress_apply(preprocess_no_NE)

In [10]:
train = queries_data[["Текст вопроса", "no_NE", "with_NE", "Номер связки\n"]][:int(len(queries_data)*0.7)]
train = train.rename(columns = {"Текст вопроса": "question", "Номер связки\n": "index"}, inplace = False)

test = queries_data[["Текст вопроса", "no_NE", "with_NE", "Номер связки\n"]][int(len(queries_data)*0.7):]
test = test.rename(columns = {"Текст вопроса": "question", "Номер связки\n": "index"}, inplace = False)

In [11]:
answers_data = pd.read_excel("answers_base.xlsx")

question_df = pd.DataFrame(columns=["index", "question", "no_NE", "with_NE"])
for question_chunk, answer_n in tqdm(answers_data[["Текст вопросов", "Номер связки"]].values):
    questions = question_chunk.split('\n')
    for q in questions:
        question_df = question_df.append({"index": answer_n,
                            "question": q,
                            "no_NE": preprocess_no_NE(q),
                            "with_NE": preprocess_with_NE(q)},
                           ignore_index=True)

In [12]:
train = train.append(question_df, sort=False, ignore_index = True)

### векторизация

In [15]:
import numpy as np

In [144]:
def normalize(v):
    return v / np.sqrt(np.sum(v ** 2))

def get_lemmas_vectors(lemm):
    lemmas = lemm.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))

    for idx, lemma in enumerate(lemmas):
        lemmas_vectors[idx] = model[lemma]
            
    return lemmas_vectors

def get_mean_vector(lemmas_vectors):
    vec = np.zeros((model.vector_size,))
    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)
        vec = normalize(vec)
    return vec

In [145]:
for df in [train, test]:
    df['vectors_no_NE'] = df['no_NE'].progress_apply(get_lemmas_vectors)
    df['mean_vector_no_NE'] = df['vectors_no_NE'].progress_apply(get_mean_vector)
    
    df['vectors_with_NE'] = df['with_NE'].progress_apply(get_lemmas_vectors)
    df['mean_vector_with_NE'] = df['vectors_with_NE'].progress_apply(get_mean_vector)

## поиск
### по векторам

In [146]:
def search_vector_no_NE(vec):
    res = np.dot(np.vstack(train['mean_vector_no_NE'].values), vec)
    index = np.argmax(res)
    return train["index"][index]

def search_vector_with_NE(vec):
    res = np.dot(np.vstack(train['mean_vector_with_NE'].values), vec)
    index = np.argmax(res)
    return train["index"][index]

In [147]:
test['pred_no_NE'] = test['mean_vector_no_NE'].progress_apply(search_vector_no_NE)

In [148]:
test['pred_with_NE'] = test['mean_vector_with_NE'].progress_apply(search_vector_with_NE)

In [149]:
(test['pred_no_NE'] == test['index']).sum() / len(test)

0.5159420289855072

In [150]:
(test['pred_with_NE'] == test['index']).sum() / len(test)

0.5420289855072464

### по матрицам

In [176]:
def search_matrix_no_NE(query):
    sims = []
    for doc in train['vectors_no_NE']:
        sim = doc.dot(query.T)
        if sim.shape[0] != 0:
            sim = np.max(sim, axis=0)
            sims.append(sim.sum())
        else:
            sims.append(0)
    index = np.argmax(sims)
    return train["index"][index]

def search_matrix_with_NE(query):
    sims = []
    for doc in train['vectors_with_NE']:
        sim = doc.dot(query.T)
        if sim.shape[0] != 0:
            sim = np.max(sim, axis=0)
            sims.append(sim.sum())
        else:
            sims.append(0)
    index = np.argmax(sims)
    return train["index"][index]

In [177]:
test['pred_matrix_no_NE'] = test['vectors_no_NE'].progress_apply(search_matrix_no_NE)

In [178]:
(test['pred_matrix_no_NE'] == test['index']).sum() / len(test)

0.4260869565217391

In [179]:
test['pred_matrix_with_NE'] = test['vectors_with_NE'].progress_apply(search_matrix_with_NE)

In [180]:
(test['pred_matrix_with_NE'] == test['index']).sum() / len(test)

0.4492753623188406


||raw|natasha|lem|
|--|------|-------|---|
tf-idf|0.475362|0.45942|0.492754
bm25|0.481159|0.514493|0.527536
matrix||0.426087|0.449275
vector||0.515942|0.542029

_Анна Полянская, БКЛ171_